In [1]:
import sys
import os
import sqlite3
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [2]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.3.2 pyspark-shell'

In [3]:
sc = SparkContext(appName='streamTest')
ssc = StreamingContext(sc, 5) 

In [4]:
def insertData(data):
    sql = "insert into samsung values(?,?)" # 튜플로 들어감
    db = sqlite3.connect(r'C:\samsung.db')
    db.execute( sql, data ) 
    db.commit()# insert, delete, update
    db.close()

In [5]:
def recvData(data):
    insertData(data) # 함수 호출
    print("data=", data) # ('키', '값') 튜플
    return data

In [6]:
topic = 'samsung'
# 내가 관심있는 토픽은 sample이다!
kvs = KafkaUtils.createStream(ssc, 'localhost', 'raw-event-streaming-consumer', {topic:1} )

#=======================================
#메시지 수신시 recvData call
lines = kvs.map( recvData ) 
lines.pprint() 
#=======================================
ssc.start() 
ssc.awaitTerminationOrTimeout(2) 

False

-------------------------------------------
Time: 2021-01-13 10:00:55
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:01:00
-------------------------------------------
('2021-01-13', '89900\n')
('2021-01-04', '83000\n')
('2021-01-05', '83900\n')
('2021-01-06', '82200\n')
('2021-01-07', '82900\n')
('2021-01-08', '88800\n')
('2021-01-11', '91000\n')
('2021-01-12', '90600\n')
('2021-01-13', '89900\n')

-------------------------------------------
Time: 2021-01-13 10:01:05
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:01:10
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:01:15
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:01:20
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:01:25
------

-------------------------------------------
Time: 2021-01-13 10:06:30
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:06:35
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:06:40
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:06:45
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:06:50
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:06:55
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:07:00
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:07:05
-------------------------------------------

-------------------------------------------
Time: 2021-01-13 10:07:10
----------

In [8]:
ssc.stop() # 폴링중지